In [49]:
import pandas as pd

'''
Более чем в 50% и до 75% случаях делали операции
Более 75% старше 6 месяцев
Средняя температура ~ 38.2
Наиболее частый пульс 48, средний 72 за счёт высоких показателе в отдельных случаях
Исходя из цвета слизистой  в ~50% не было серьёзных нарушений, где-то 25% начальный шок и 25% имело серьёзные проблемы 
включая сепсис 
Более чем у половины не было серьёзного вздутия живота где-то от 25% имели такую проблему
Примерно в 50% случаев были проблемы с толстым и тонким кишечником требующие хирургического вмешательства
Более 50% лошадей выжали, примерно 25% погибли часть из них вероятно из-за сепсиса
'''

horse_df = pd.read_csv("horse_data.csv",
                   na_values="?",
                   usecols=[0,1,3,4,8,12,17,22],
                   names=["surgery", "age", "rectal_temp", "pulse", "muc_membran", "abd_distension","abdomen","outcome"])
modes={col:horse_df[col].round().mode()[0] for col in horse_df.columns}
horse_df_desc = horse_df.describe()
m_series = pd.Series(data=modes, name="mode")
horse_df_desc = horse_df_desc.append(m_series, ignore_index=False)
print(horse_df_desc)


          surgery         age  rectal_temp       pulse  muc_membran  \
count  299.000000  300.000000   240.000000  276.000000   253.000000   
mean     1.397993    1.640000    38.167917   71.913043     2.853755   
std      0.490305    2.173972     0.732289   28.630557     1.620294   
min      1.000000    1.000000    35.400000   30.000000     1.000000   
25%      1.000000    1.000000    37.800000   48.000000     1.000000   
50%      1.000000    1.000000    38.200000   64.000000     3.000000   
75%      2.000000    1.000000    38.500000   88.000000     4.000000   
max      2.000000    9.000000    40.800000  184.000000     6.000000   
mode     1.000000    1.000000    38.000000   48.000000     1.000000   

       abd_distension     abdomen     outcome  
count      244.000000  182.000000  299.000000  
mean         2.266393    3.692308    1.551839  
std          1.065131    1.491725    0.737187  
min          1.000000    1.000000    1.000000  
25%          1.000000    2.000000    1.000000  
5

In [51]:

'''
выбросы по  температуре и пульсу в пределах допустимых значений, я бы оставил всё как есть.
Разве что в строки 259 у лошади температура 40.8   при относительно нормальных показателях по другим параметрам,
хотя пульс повышен. Тут или не хорактерное заболевание для диагностики которого надо ввести дополнительные параметры,
либо строку с этим выбросом стоит удалить.

'''

horse_df = pd.read_csv("horse_data.csv",
                   na_values="?",
                   usecols=[0,1,3,4,8,12,17,22],
                   names=["surgery", "age", "rectal_temp", "pulse", "muc_membran", "abd_distension","abdomen","outcome"])

def rem_outliers(col):
    q1 = col.quantile(0.25)
    q3 = col.quantile(0.75)
    iqr = q3 - q1
    lower_bound_tempr = q1 - (1.5 * iqr) 
    upper_bound_tempr = q3 + (1.5 * iqr)
    return col.between(lower_bound_tempr, upper_bound_tempr)

print("#### temperature column outliers")
remove_outliers = horse_df[rem_outliers(horse_df["rectal_temp"])]
print(pd.concat([horse_df, remove_outliers]).drop_duplicates(keep=False)[horse_df["rectal_temp"].notna()]["rectal_temp"])
print("#### pulse column outliers")
remove_outliers = horse_df[rem_outliers(horse_df["pulse"])]
print(pd.concat([horse_df, remove_outliers]).drop_duplicates(keep=False)[horse_df["pulse"].notna()]["pulse"])

#### temperature column outliers
20     39.9
44     35.4
54     40.3
75     39.7
80     36.4
91     40.3
99     39.6
118    36.5
141    36.0
238    36.1
251    36.6
259    40.8
281    40.0
298    36.5
Name: rectal_temp, dtype: float64
#### pulse column outliers
3      164.0
41     150.0
55     160.0
255    184.0
275    150.0
Name: pulse, dtype: float64


<ipython-input-51-fc1f7ede3e27>:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(pd.concat([horse_df, remove_outliers]).drop_duplicates(keep=False)[horse_df["rectal_temp"].notna()]["rectal_temp"])
<ipython-input-51-fc1f7ede3e27>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(pd.concat([horse_df, remove_outliers]).drop_duplicates(keep=False)[horse_df["pulse"].notna()]["pulse"])


In [52]:
for col in horse_df.columns:
    pct_missing = horse_df[col].isnull().mean()
    print(f'{col} - {pct_missing :.1%}')

surgery - 0.3%
age - 0.0%
rectal_temp - 20.0%
pulse - 8.0%
muc_membran - 15.7%
abd_distension - 18.7%
abdomen - 39.3%
outcome - 0.3%


In [54]:
# fill_median.Age.fillna(titanic.Age.median(), inplace=True)
'''
Замена предыдущим или последующим значением  не релевантна так как каждая строка описывает случай не зависящий от остальных
Буду восстанавливать данные с помощью последовательной группировки по разным связанным столбцам, так как не всегда 
получается заполнить все пропуске с первого раза. Возможно группировка по нескольким столбцам одновременно даёт более точный результат
но и оставляет большее количество пробелов, а в случае моды не позволяет вычислить этот показатель на данном наборе данных

surgery - категориальная величина часто зависящая от состояния желудка, буду использовать моду и группировку по столбцам 
описывающих результат лечения

rectal_temp - числовая величина буду использовать медиану как более устойчивую к выбросам, есть зависимость от пульса и 
цвета слизистой как показателя отображающего общее состояние, плюс возраст

pulse - числовая величина описывающая общее состояние. Буду использовать медиану и группировку по температуре и состоянию 
слизистой, так как эти величины так же описывают общее состояние, плюс возраст 

muc_membran - категориальная величина, мода. Описывает общее состояния лошади,  возможно коррелирует с результатом лечения

abd_distension категориальная величина, мода. Думаю коррелирует с цветом слизистой.

abdomen - категориальная величина, мода. Коррелирует со степенью вздутия желудка и думаю с цветом слизистой.

outcome -  категориальная величина, мода. Думаю есть зависимость от возраста.  Возможно было бы лучше группировать по 
наличию операции, но такая группировка не позволяет вычислить моду

'''
horse_df = pd.read_csv("horse_data.csv",
                   na_values="?",
                   usecols=[0,1,3,4,8,12,17,22],
                   names=["surgery", "age", "rectal_temp", "pulse", "muc_membran", "abd_distension","abdomen","outcome"])

print(horse_df.describe())

#удалить строки у которых заполненно 3 значения из 8
horse_df = horse_df.dropna(thresh=3)

horse_df.outcome.fillna(horse_df.groupby(["age"]).outcome.transform(lambda x: x.mode()[0]), inplace=True)
horse_df.surgery.fillna(horse_df.groupby(['outcome']).surgery.transform(lambda x: x.mode()[0]), inplace=True)

horse_df.rectal_temp.fillna(horse_df.groupby(['pulse']).rectal_temp.transform('median'), inplace=True)
horse_df.rectal_temp.fillna(horse_df.groupby(['muc_membran']).rectal_temp.transform('median'), inplace=True)
horse_df.rectal_temp.fillna(horse_df.groupby(['age']).rectal_temp.transform('median'), inplace=True)

horse_df.pulse.fillna(horse_df.groupby(['rectal_temp']).pulse.transform('median'), inplace=True)
horse_df.pulse.fillna(horse_df.groupby(["muc_membran"]).pulse.transform('median'), inplace=True)
horse_df.pulse.fillna(horse_df.groupby(['age']).pulse.transform('median'), inplace=True)

horse_df.muc_membran.fillna(horse_df.groupby(["outcome"]).muc_membran.transform(lambda x: x.mode()[0]), inplace=True)
horse_df.abd_distension.fillna(horse_df.groupby(["muc_membran"]).abd_distension.transform(lambda x: x.mode()[0]), inplace=True)
horse_df.abdomen.fillna(horse_df.groupby(["abd_distension"]).abdomen.transform(lambda x: x.mode()[0]), inplace=True)


for col in horse_df.columns:
    pct_missing = horse_df[col].isnull().mean()
    print(f'{col} - {pct_missing :.1%}')
  
'''
ну вроде не плохо получилось ), основные показатели изменились не значительно, средне квадратическое отклонение немного
уменьшилось
'''

horse_df.describe()


          surgery         age  rectal_temp       pulse  muc_membran  \
count  299.000000  300.000000   240.000000  276.000000   253.000000   
mean     1.397993    1.640000    38.167917   71.913043     2.853755   
std      0.490305    2.173972     0.732289   28.630557     1.620294   
min      1.000000    1.000000    35.400000   30.000000     1.000000   
25%      1.000000    1.000000    37.800000   48.000000     1.000000   
50%      1.000000    1.000000    38.200000   64.000000     3.000000   
75%      2.000000    1.000000    38.500000   88.000000     4.000000   
max      2.000000    9.000000    40.800000  184.000000     6.000000   

       abd_distension     abdomen     outcome  
count      244.000000  182.000000  299.000000  
mean         2.266393    3.692308    1.551839  
std          1.065131    1.491725    0.737187  
min          1.000000    1.000000    1.000000  
25%          1.000000    2.000000    1.000000  
50%          2.000000    4.000000    1.000000  
75%          3.000000   

,surgery,age,rectal_temp,pulse,muc_membran,abd_distension,abdomen,outcome
count,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000
mean,1.396667,1.640000,38.177417,71.893333,2.716667,2.163333,3.410000,1.550000
std,0.490023,2.173972,0.665364,27.997765,1.597361,1.058485,1.666681,0.736642
min,1.000000,1.000000,35.400000,30.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,37.875000,48.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,38.150000,64.000000,3.000000,2.000000,4.000000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,4.000000,3.000000,5.000000,2.000000
max,2.000000,9.000000,40.800000,184.000000,6.000000,4.000000,5.000000,3.000000
